In [1]:
from pipeline.train_model import get_token_data

In [2]:
X, y, tokenizer = get_token_data()

Pandas Apply:   0%|          | 0/310217 [00:00<?, ?it/s]

In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
X.shape

torch.Size([310217, 80])

In [5]:
set(y.tolist())

{0, 1, 2}

In [6]:
n_embd = 24
dropout = 0.2

class SelfAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        T, C, _ = x.shape

        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        wei = q @ k.transpse()

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = wei @ v

        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads

    def forward(self):
        pass


In [7]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(X, y)

# Split into train/test (80/20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)


In [8]:
class SimpleTextNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, pad_idx):
        super(SimpleTextNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embed_dim, num_classes)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        embedded = self.embedding(x)          # (batch_size, seq_len, embed_dim)
        pooled = embedded.mean(dim=1)         # mean pooling: (batch_size, embed_dim)
        out = self.fc(pooled)                 # (batch_size, num_classes)
        return out


In [9]:
vocab_size = tokenizer.get_vocab_size()
embed_dim = 128
num_classes = len(y.unique())
pad_idx = tokenizer.token_to_id("<pad>")

model = SimpleTextNN(vocab_size, embed_dim, num_classes, pad_idx)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SimpleTextNN(
  (embedding): Embedding(30000, 128, padding_idx=0)
  (fc): Linear(in_features=128, out_features=3, bias=True)
)

In [10]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/5, Loss: 0.3891
Epoch 2/5, Loss: 0.3085
Epoch 3/5, Loss: 0.2926
Epoch 4/5, Loss: 0.2827
Epoch 5/5, Loss: 0.2752


In [11]:
import torch
import torch.nn.functional as F

def predict_text(model, tokenizer, text, max_len=80, device='cpu'):

    model.eval()
    
    # Encode + pad
    ids = tokenizer.encode(text).ids
    if len(ids) < max_len:
        PAD_ID = tokenizer.token_to_id("<pad>")
        ids += [PAD_ID] * (max_len - len(ids))
    ids = ids[:max_len]
    
    # Convert to tensor
    x = torch.tensor([ids], dtype=torch.long).to(device)
    
    # Forward pass
    with torch.no_grad():
        logits = model(x)
        probs = F.softmax(logits, dim=1)  # convert to probabilities
    
    return probs.squeeze()  # remove batch dimension


In [12]:
text = "Rahmat, juda yoqdi!"
probs = predict_text(model, tokenizer, text, max_len=80, device=device)
print(probs)              # tensor with class probabilities
print(torch.argmax(probs)) # predicted class index


tensor([0.0095, 0.0171, 0.9734])
tensor(2)


In [16]:
text = "bomapti"
probs = predict_text(model, tokenizer, text, max_len=80, device=device)
print(probs)              # tensor with class probabilities
print(torch.argmax(probs)) # predicted class index

tensor([0.9032, 0.0746, 0.0222])
tensor(0)
